**Assignment includes the following points(SVM, Desicion Tree , Random Forest and Logistic Regression)**
- Train-test Split Model Training and Evaluation
- Cross Validation
- Select Best Performing Model Using CV
- Fine Tune That Model
- Evaluate Again
- Apply Ada Boost On Fine Tune Model

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
warnings.filterwarnings('ignore')

#  Load Dataset


In [2]:
df=pd.read_csv('train.csv')

 **Used to get first 5 rows of Dataset** 

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Obtain Shape of dataset**

In [4]:
df.shape

(891, 12)

**Observation**
- 891 rows
- 12 columns

**Check Datatype of Indiviual column**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Overview Data**

In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Observation
- Describe function returns the statistical summary of the dataframe or series.
- This includes count, mean, median (or 50th percentile) standard variation, min-max, and percentile values of columns. 

**For check NaN values in dataset**

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

Cabin          77.104377
Age            19.865320
Embarked        0.224467
Fare            0.000000
Ticket          0.000000
Parch           0.000000
SibSp           0.000000
Sex             0.000000
Name            0.000000
Pclass          0.000000
Survived        0.000000
PassengerId     0.000000
dtype: float64

**Observations:**
- Cabin==77.1% NaN values
- Age==19.9% NaN values
- Embarked==0.22% NaN values

**Lets extract the Salutations**

In [9]:
df['Initial']=0
for i in df:
    df['Initial']=df.Name.str.extract('([A-Za-z]+)\.') 

In [10]:
df['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

**Lets check the average age by Initials**

In [11]:
df.groupby('Initial')['Age'].mean()

Initial
Master     4.574167
Miss      21.860000
Mr        32.739609
Mrs       35.981818
Other     45.888889
Name: Age, dtype: float64

# Filling NaN Ages

**Assigning the NaN Values with the Ceil values of the mean ages**

In [12]:
df.loc[(df.Age.isnull())&(df.Initial=='Mr'),'Age']=33
df.loc[(df.Age.isnull())&(df.Initial=='Mrs'),'Age']=36
df.loc[(df.Age.isnull())&(df.Initial=='Master'),'Age']=5
df.loc[(df.Age.isnull())&(df.Initial=='Miss'),'Age']=22
df.loc[(df.Age.isnull())&(df.Initial=='Other'),'Age']=46

**All NaN values fill finally**

In [13]:
df.Age.isnull().sum()

0

In [14]:
ndf['Cabin'] = df['Cabin'].apply(lambda x:not isinstance(x, float))

In [15]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Initial'],
      dtype='object')

# Drop Columns 

**Drop unnecessary column**

In [16]:
df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [17]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Initial
0,0,3,male,22.0,1,0,7.2500,False,S,Mr
1,1,1,female,38.0,1,0,71.2833,True,C,Mrs
2,1,3,female,26.0,0,0,7.9250,False,S,Miss
3,1,1,female,35.0,1,0,53.1000,True,S,Mrs
4,0,3,male,35.0,0,0,8.0500,False,S,Mr


Columns drop successfully.

# Label Encoding

In [18]:
label_encoder={}
for x in ['Sex']:
    label_encoder[x]=LabelEncoder()
    df[x]=label_encoder[x].fit_transform(df[x])

In [19]:
 label_encoder={}
for x in ['Embarked']:
    label_encoder[x]=LabelEncoder()
    df[x]=label_encoder[x].fit_transform(df[x].astype(str))

In [20]:
label_encoder={}
for x in ['Initial']:
    label_encoder[x]=LabelEncoder()
    df[x]=label_encoder[x].fit_transform(df[x])

# Split the feature and target output


In [21]:
x=df[df.columns[1:]].values
y=df.Survived

**Obesarvation:**
- We train the model using the training set and then apply the model to the test set.  In this way, we can evaluate the performance of our model.

# For train_test_ split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42,shuffle=True)
#Split into train test sets

# SVM Model

In [23]:
svm=SVC()
#fit the model
svm.fit(X_train,y_train)

SVC()

In [24]:
#make predictions
y_pred=svm.predict(X_test)

In [25]:
#evaluate predictions
print('Accuracy=',accuracy_score(y_test,y_pred))
print("Precision=",precision_score(y_test,y_pred))
print("Recall_Score=",recall_score(y_test,y_pred))
print("f1_score=",f1_score(y_test,y_pred))
print("ROC_AUC=",roc_auc_score(y_test,y_pred))

Accuracy= 0.6536312849162011
Precision= 0.75
Recall_Score= 0.24324324324324326
f1_score= 0.3673469387755103
ROC_AUC= 0.593050193050193


# Decision Tree Model

In [26]:
dt=DecisionTreeClassifier()

In [27]:
#fit the model
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [28]:
#make predictions
y_pred=dt.predict(X_test)

In [29]:
#evaluate predictions
print('Accuracy=',accuracy_score(y_test,y_pred))
print("Precision=",precision_score(y_test,y_pred))
print("Recall_Score=",recall_score(y_test,y_pred))
print("f1_score=",f1_score(y_test,y_pred))
print("ROC_AUC=",roc_auc_score(y_test,y_pred))

Accuracy= 0.8100558659217877
Precision= 0.7857142857142857
Recall_Score= 0.7432432432432432
f1_score= 0.7638888888888888
ROC_AUC= 0.8001930501930502


# Random Forest Model

In [30]:
rf=RandomForestClassifier()

In [31]:
#fit the model
rf.fit(X_train,y_train)

RandomForestClassifier()

In [32]:
#make predictions
y_pred=rf.predict(X_test)

In [33]:
#evalute predictions
print('Accuracy=',accuracy_score(y_test,y_pred))
print("Precision=",precision_score(y_test,y_pred))
print("Recall_Score=",recall_score(y_test,y_pred))
print("f1_score=",f1_score(y_test,y_pred))
print("ROC_AUC=",roc_auc_score(y_test,y_pred))

Accuracy= 0.8324022346368715
Precision= 0.7972972972972973
Recall_Score= 0.7972972972972973
f1_score= 0.7972972972972974
ROC_AUC= 0.8272200772200773


# Logistic Regression Model

In [34]:
#fit the model
lr=LogisticRegressionCV()
lr.fit(X_train,y_train)

LogisticRegressionCV()

In [35]:
#make predictions
y_pred=lr.predict(X_test)

In [36]:
#evaluate predictions
print('Accuracy=',accuracy_score(y_test,y_pred))
print("Precision=",precision_score(y_test,y_pred))
print("Recall_Score=",recall_score(y_test,y_pred))
print("f1_score=",f1_score(y_test,y_pred))
print("ROC_AUC=",roc_auc_score(y_test,y_pred))

Accuracy= 0.8212290502793296
Precision= 0.8
Recall_Score= 0.7567567567567568
f1_score= 0.7777777777777778
ROC_AUC= 0.8117117117117117


# Result

**Accuracy**
- Random Forest Model **>** Logistic Regression Model **>**  Decision Tree Model **>** Decision Tree Model
- In train_test_split **Random Forest Model** show **highest** accuracy **(83%)**.

# For Cross Validation

**Using K-Folds**

In [37]:
def Cross_Validation_Model(x,y,k_fold_validation=KFold(10)):
    md=LogisticRegressionCV(random_state=42)
    conclusion=cross_val_score(md,x,y,cv=k_fold_validation)
    print('LogisticRegression Model Kfold 10 Value')
    print(' ')
    print(conclusion)
    print("The Accuracy of LogisticRegressionCV :{}".format(np.mean(conclusion)*100))
    print("     ")
    
    md=DecisionTreeClassifier(random_state=42)
    conclusion=cross_val_score(md,x,y,cv=k_fold_validation)
    print('DecisionTreeClassifier Model Kfold 10 Value')
    print(' ')
    print(conclusion)
    print("The Accuracy of DecisionTreeClassifier :{}".format(np.mean(conclusion)*100))
    print("     ") 
    
    md=SVC(random_state=42)
    conclusion=cross_val_score(md,x,y,cv=k_fold_validation)
    print('SVC Model Kfold 10 Value')
    print(' ')
    print(conclusion)
    print("The Accuracy of SVC :{}".format(np.mean(conclusion)*100))
    print("     ")
    
    md=RandomForestClassifier(random_state=42)
    conclusion=cross_val_score(md,x,y,cv=k_fold_validation)
    print('RandomForestClassifier Model Kfold 10 Value')
    print(' ')
    print(conclusion)
    print("The Accuracy of RandomForestClassifier :{}".format(np.mean(conclusion)*100))
    print("     ")

In [38]:
Cross_Validation_Model(x,y)

LogisticRegression Model Kfold 10 Value
 
[0.81111111 0.84269663 0.7752809  0.78651685 0.80898876 0.76404494
 0.76404494 0.79775281 0.82022472 0.84269663]
The Accuracy of LogisticRegressionCV :80.13358302122346
     
DecisionTreeClassifier Model Kfold 10 Value
 
[0.72222222 0.73033708 0.68539326 0.79775281 0.75280899 0.82022472
 0.75280899 0.74157303 0.83146067 0.86516854]
The Accuracy of DecisionTreeClassifier :76.9975031210986
     
SVC Model Kfold 10 Value
 
[0.6        0.6741573  0.6741573  0.6741573  0.65168539 0.70786517
 0.65168539 0.73033708 0.73033708 0.68539326]
The Accuracy of SVC :67.79775280898878
     
RandomForestClassifier Model Kfold 10 Value
 
[0.81111111 0.85393258 0.75280899 0.82022472 0.83146067 0.82022472
 0.76404494 0.78651685 0.85393258 0.88764045]
The Accuracy of RandomForestClassifier :81.81897627965044
     


# Result

**Accuracy**
- RandomForestClassifier **>** LogisticRegression **>** DecisionTreeClassifier **>** SVC
- **RandomForestClassifier** shows highest accuracy in Cross validation using K-folds : **(81.8% approximately equal to 82%)**.

**So, best performing Model (Random Forest Classifier) is used for Fine Tune.**

In [39]:
kvalue=KFold(10)
md=RandomForestClassifier(random_state=42)
conclusion=cross_val_score(md,x,y,cv=kvalue)

In [40]:
np.mean(conclusion)

0.8181897627965045

# Random Search CV

In [41]:
# Number of trees in random forest
n_estimators = [40,80,120,160]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [2,4,8]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [42]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [40, 80, 120, 160], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4, 8, None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'bootstrap': [True, False]}


**Observation:**
- On each iteration, the algorithm will choose a difference combination of the features.However, the benefit of a random search is that we are not trying every combination, but selecting at random to sample a wide range of values.

In [43]:
# Use the random grid to search for best hyperparameters
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 4, verbose=2, random_state=42, n_jobs = -1)

In [44]:
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   52.0s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   57.0s finished


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 4, 8, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 3, 4],
                                        'n_estimators': [40, 80, 120, 160]},
                   random_state=42, verbose=2)

In [45]:
rf_random.best_params_

{'n_estimators': 160,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 4,
 'bootstrap': False}

In [46]:
rf_random.best_score_*100

83.98876404494382

**Observation**
- Performance on HypertunningModel is **83.9%**  approximately **84%** which is **2%** better.

# Evaluate Random Search

In [47]:
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Model Performance')
    print('Accuracy: {:0.5f}'.format(accuracy*100))
    print('F1 Score: {:0.5f}'.format(f1*100))
    
    return [accuracy, f1]

**Best Model Score**

In [48]:
best_random = rf_random.best_estimator_
random_score = evaluate(best_random, X_test, y_test)

Model Performance
Accuracy: 81.00559
F1 Score: 75.71429


# AdaBoost Classifier in Python

In [49]:
rf=RandomForestClassifier(random_state=42)

In [50]:
abc = AdaBoostClassifier(n_estimators=100, base_estimator=lr, learning_rate=1)

In [51]:
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)

In [54]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8212290502793296


**Observation**
- **Adaboost shows Accuracy=82%**